# 3D Hand-writting test

In [1]:
# Import Numpy, TensorFlow, and MNIST data
import numpy as np
import tensorflow as tf
import tflearn.datasets.mnist as mnist
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pickle

In [2]:
# Retrieve the training and test data
trainX, trainY, testX, testY = mnist.load_data(one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


## Visualize the training data

# Visualizing the data
import matplotlib.pyplot as plt
%matplotlib inline

def display_same_digit(digit,number):
    collectionY = []
    for i in range(0,len(trainY)):
        if trainY[i][digit]==1:
            collectionY.append(i)
    collectionX = [trainX[i] for i in collectionY]
    for j in range(number):
        image = collectionX[j].reshape([28,28])
        plt.title('Training data, Label: %d' % digit)
        plt.imshow(image, cmap='gray_r')
        plt.show()
        
display_same_digit(6,20)

# Make 3d digit from 2d image

#convert training data

def conv_2d_3d(digit_2d):
    digit_3d=[]
    for i in range(len(digit_2d)):
        single_3d=np.zeros([28,28,28],dtype=np.bool_)
        if i%500==0:
            print(i)
        for j in range(28):
            for k in range(28):
                depth =int(np.ceil(digit_2d[i][j][k]*4))
                single_3d[j][k][14-depth:13+depth]=True
        digit_3d.append(single_3d)
    return digit_3d

#converting testing data
train_digit_2d = trainX.reshape([len(trainX),28,28])
test_digit_2d = testX.reshape([len(testX),28,28])
train_digit_3d=conv_2d_3d(train_digit_2d)
test_digit_3d=conv_2d_3d(test_digit_2d)


np.save('train_digit_3d.out', train_digit_3d) 
np.save('test_digit_3d.out', test_digit_3d) 

from mpl_toolkits.mplot3d import Axes3D

def show_2d(digit_2d, targets,index):
    plt.title('Label: %d' % np.argmax(targets[index]))
    plt.imshow(digit_2d[index], cmap='gray_r')
    plt.show()
show_2d(train_digit_2d, trainY, 2)

def show_3d(digit_3d, targets,index):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    pos = np.where(digit_3d[index]==1)
    ax.scatter(pos[0], pos[1], pos[2],c='black')
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    axes = plt.gca()
    axes.set_xlim([0,27])
    axes.set_ylim([0,27])
    axes.set_zlim([0,27])
    for angle in range(0, 360):
        ax.view_init(60, angle)
        plt.draw()
        plt.pause(.001)

show_3d(train_digit_3d, trainY, 2)


In [3]:
test_digit=np.load('test_digit_3d.out.npy')
test_target=testY

whole_digit=np.load('train_digit_3d.out.npy')
divide=int(len(whole_digit)*0.9)
whole_target=trainY
train_digit=whole_digit[0:divide]
train_target=whole_target[0:divide]
valid_digit=whole_digit[divide+1:]
valid_target=whole_target[divide+1:]
print(len(train_digit))

49500


In [4]:
def neural_net_image_input(image_shape):
    """
    Return a Tensor for a bach of image input
    : image_shape: Shape of the images
    : return: Tensor for image input.
    """
    # TODO: Implement Function
    return tf.placeholder(tf.float32,shape=[None, image_shape[0], image_shape[1], image_shape[2],1],name = "x")


def neural_net_label_input(n_classes):
    """
    Return a Tensor for a batch of label input
    : n_classes: Number of classes
    : return: Tensor for label input.
    """
    # TODO: Implement Function
    return tf.placeholder(tf.float32,shape=[None, n_classes],name = "y")


def neural_net_keep_prob_input():
    """
    Return a Tensor for keep probability
    : return: Tensor for keep probability.
    """
    # TODO: Implement Function
    keep_prob=tf.placeholder(tf.float32, name= "keep_prob" )
    return keep_prob

In [5]:
def conv3d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
    """
    Apply convolution then max pooling to x_tensor
    :param x_tensor: TensorFlow Tensor
    :param conv_num_outputs: Number of outputs for the convolutional layer
    :param conv_ksize: kernal size 3-D Tuple for the convolutional layer
    :param conv_strides: Stride 3-D Tuple for convolution
    :param pool_ksize: kernal size 3-D Tuple for pool
    :param pool_strides: Stride 3-D Tuple for pool
    : return: A tensor that represents convolution and max pooling of x_tensor
    """
    # TODO: Implement Function
    weight = tf.Variable(tf.random_normal([conv_ksize[0], conv_ksize[1],conv_ksize[2], x_tensor.get_shape().as_list()[-1], conv_num_outputs],mean=0.0,stddev = 0.1)) 
    
    conv_layer = tf.nn.conv3d(
        x_tensor, 
        weight, 
        strides=[1, conv_strides[0], conv_strides[1],conv_strides[2], 1], 
        padding='SAME')    
    
    conv_layer = tf.nn.relu(conv_layer)                           
    out=tf.nn.max_pool3d(
        conv_layer,
        ksize=[1, pool_ksize[0], pool_ksize[1], pool_ksize[2], 1],
        strides=[1, pool_strides[0], pool_strides[1], pool_strides[2], 1],
        padding='SAME')
    return out

In [6]:
def flatten(x_tensor):
    """
    Flatten x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    # TODO: Implement Function
    shape=x_tensor.get_shape().as_list()
    return tf.reshape(x_tensor, [-1,shape[1]*shape[2]*shape[3]*shape[4]])

In [7]:
def fully_conn(x_tensor, num_outputs):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    shape=x_tensor.get_shape().as_list()
    weight = tf.Variable(tf.random_normal([shape[1],num_outputs],mean=0.0,stddev = 0.1))
    bias = tf.Variable(tf.random_normal([num_outputs],mean=0.0,stddev = 0.1))
    output = tf.add(tf.matmul(x_tensor, weight), bias)
    output = tf.nn.relu(output)
    return output

In [8]:
def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    shape=x_tensor.get_shape().as_list()
    weight = tf.Variable(tf.random_normal([shape[1],num_outputs],mean=0.0,stddev = 0.1))
    bias = tf.Variable(tf.random_normal([num_outputs],mean=0.0,stddev = 0.1))
    output = tf.add(tf.matmul(x_tensor, weight), bias)
    return output

In [9]:
def conv_net(x, keep_prob):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
    # TODO: Apply 1, 2, or 3 Convolution and Max Pool layers
    #    Play around with different number of outputs, kernel size and stride
    # Function Definition from Above:
    #    conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
    conv_num_outputs1=32
    conv_num_outputs2=64
    conv_ksize=[5,5,5]
    conv_strides=[2,2,2]
    pool_ksize=[2,2,2]
    pool_strides=[2,2,2]
    
    conv1=conv3d_maxpool(x, conv_num_outputs1, conv_ksize, conv_strides, pool_ksize, pool_strides)
    conv2=conv3d_maxpool(conv1, conv_num_outputs2, conv_ksize, conv_strides, pool_ksize, pool_strides)

    # TODO: Apply a Flatten Layer
    # Function Definition from Above:
    #   flatten(x_tensor)
    ft=flatten(conv2)

    # TODO: Apply 1, 2, or 3 Fully Connected Layers
    #    Play around with different number of outputs
    # Function Definition from Above:
    #   fully_conn(x_tensor, num_outputs)
    num_outputs1=64
    num_outputs2=16
    fc1=fully_conn(ft, num_outputs1)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    fc2=fully_conn(fc1, num_outputs2)

    
    # TODO: Apply an Output Layer
    #    Set this to the number of classes
    # Function Definition from Above:
    #   output(x_tensor, num_outputs)
    out=output(fc2, 10)
    # TODO: return output
    return out



##############################
## Build the Neural Network ##
##############################

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = neural_net_image_input((28, 28, 28, 1))
y = neural_net_label_input(10)
keep_prob = neural_net_keep_prob_input()

# Model
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [10]:
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    # TODO: Implement Function
    loss = session.run(cost, feed_dict = 
                   {x: feature_batch,y: label_batch, keep_prob: 1.0})
    valid_digit2=np.expand_dims(valid_digit,4)
    acc = session.run(accuracy,feed_dict = 
                  {x: valid_digit2[0:2048], y: valid_target[0:2048], keep_prob: 1.0})
    print('Loss: {:>10.4f} Accuracy: {:.6f}'.format(loss,acc))
    pass

In [11]:
# TODO: Tune Parameters
epochs = 100
batch_size = 100
keep_probability = 0.5

In [ ]:
def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    """
    Optimize the session on a batch of images and labels
    : session: Current TensorFlow session
    : optimizer: TensorFlow optimizer function
    : keep_probability: keep probability
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    """
    # TODO: Implement Function
    session.run(optimizer, feed_dict={
                x: feature_batch,
                y: label_batch,
                keep_prob: keep_probability})

def load_preprocess_training_batch(batch_id, batch_size):
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    n_batches=len(test_digit)//batch_size
    test_digit2=np.expand_dims(test_digit,4)
    digits=[]
    targets=[]
    for i in range(n_batches):
        digits.append(test_digit2[i*batch_size:(i+1)*batch_size])
        targets.append(test_target[i*batch_size:(i+1)*batch_size])
    digits2=np.array(digits)
    targets2=np.array(targets)
    return(digits2[batch_id],targets2[batch_id])

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
print('Checking the Training on a Single Batch...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    n_batches=100
    # Training cycle
    for epoch in range(epochs):
        for batch_i in range(n_batches):
            digits,targets=load_preprocess_training_batch(batch_i, batch_size)
            train_neural_network(sess, optimizer, keep_probability,digits, targets)
        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
        print_stats(sess, digits, targets, cost, accuracy)

Checking the Training on a Single Batch...
Epoch  1, CIFAR-10 Batch 99:  Loss:     0.7432 Accuracy: 0.752441
Epoch  2, CIFAR-10 Batch 99:  Loss:     0.4526 Accuracy: 0.825195
Epoch  3, CIFAR-10 Batch 99:  Loss:     0.3629 Accuracy: 0.898926
Epoch 15, CIFAR-10 Batch 99:  Loss:     0.0096 Accuracy: 0.973633
Epoch 16, CIFAR-10 Batch 99:  Loss:     0.0078 Accuracy: 0.968262
Epoch 17, CIFAR-10 Batch 99:  Loss:     0.0122 Accuracy: 0.973633
Epoch 18, CIFAR-10 Batch 99:  Loss:     0.0095 Accuracy: 0.967773
Epoch 19, CIFAR-10 Batch 99:  Loss:     0.0086 Accuracy: 0.971191
Epoch 20, CIFAR-10 Batch 99:  Loss:     0.0056 Accuracy: 0.969238
Epoch 21, CIFAR-10 Batch 99:  Loss:     0.0134 Accuracy: 0.970215
Epoch 22, CIFAR-10 Batch 99:  Loss:     0.0066 Accuracy: 0.971191
Epoch 23, CIFAR-10 Batch 99:  Loss:     0.0047 Accuracy: 0.972656
Epoch 36, CIFAR-10 Batch 99:  Loss:     0.0005 Accuracy: 0.975586
Epoch 37, CIFAR-10 Batch 99:  Loss:     0.0003 Accuracy: 0.974609
Epoch 38, CIFAR-10 Batch 99:  Los